In [4]:
%load_ext autoreload
%autoreload 2

import os
import time
import sys

from utilities2014 import *

sys.path.append('/home/yuncong/project/opencv-2.4.9/release/lib/python2.7/site-packages')
import cv2

from joblib import Parallel, delayed

from collections import defaultdict, Counter
from itertools import combinations, chain, product

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import cPickle as pickle

In [5]:
from enum import Enum

class PolygonType(Enum):
    CLOSED = 'closed'
    OPEN = 'open'
    TEXTURE = 'textured'
    TEXTURE_WITH_CONTOUR = 'texture with contour'
    DIRECTION = 'directionality'

In [6]:
thetas = np.linspace(-np.pi/4, np.pi/4, 9)
n_theta = len(thetas)
Rs = [np.array([[np.cos(theta), np.sin(theta)], [-np.sin(theta), np.cos(theta)]]) for theta in thetas]

In [7]:
landmark_indices = range(17)

In [15]:
vertices_name = {PolygonType.OPEN: 'boundary_vertices_global',
PolygonType.CLOSED: 'boundary_vertices_global',
PolygonType.DIRECTION: 'striation_points_global',
PolygonType.TEXTURE: 'textured_area_vertices_global',
PolygonType.TEXTURE_WITH_CONTOUR: 'textured_area_vertices_global'
}

In [18]:
landmark_polygons = {}
for lm_ind in landmark_indices:
    print lm_ind
    
    with open('/home/yuncong/csd395/landmark_descriptors/RS141_0001_yuncong_08212015001655_landmark_descriptors_%d.pkl'%lm_ind, 'r') as f:
        landmark_polygons[lm_ind] = pickle.load(f)[4]['polygons']
    
#     landmark_polygons[lm_ind] = [(p['type'], p[vertices_name[p['type']]]) for p in polygons]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [20]:
dm = DataManager(generate_hierarchy=False, stack='RS141', resol='x5', section=1)
dm._load_image()

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/skimage/filter/__init__.py:6: skimage_deprecation: The `skimage.filter` module has been renamed to `skimage.filters`.  This placeholder module will be removed in v0.13.
  warn(skimage_deprecation('The `skimage.filter` module has been renamed '


In [21]:
colors = (np.loadtxt(os.environ['GORDON_REPO_DIR'] + '/visualization/100colors.txt', skiprows=1) * 255).astype(np.int)

In [25]:
import cv2

def visualize_atlas(landmark_polygons):
    '''
    Visualize the atlas with landmarks in original orientation
    '''
    
    canvas_size = np.load(os.environ['GORDON_RESULT_DIR'] + '/paddedSize.npy')
    origin_on_canvas = dm.load_pipeline_result('originPosOnCanvas', 'npy')
    
    viz = 255 * np.ones(np.r_[canvas_size, 3], np.uint8)
    viz[::20, :] = 0
    viz[:, ::20] = 0

    for label, polygons in landmark_polygons.iteritems():
        for polygon in polygons:
            ptype = polygon['type']
            
            if ptype == PolygonType.OPEN or ptype == PolygonType.CLOSED:
                pts_canvas = polygon['boundary_vertices_global'] + origin_on_canvas
                for x,y in pts_canvas.astype(np.int):
                    cv2.circle(viz, (x,y), 5, colors[label], -1)

            elif ptype == PolygonType.TEXTURE or ptype == PolygonType.TEXTURE_WITH_CONTOUR:
                pts_canvas = polygon['textured_area_vertices_global'] + origin_on_canvas
                for x,y in pts_canvas.astype(np.int):
                    cv2.circle(viz, (x,y), 5, colors[label], -1)
    #             viz[pts_canvas[:,1], pts_canvas[:,0]] = (0,255,0)

            elif ptype == PolygonType.DIRECTION:
                pts_canvas = polygon['striation_points_global'] + origin_on_canvas

                vecs = polygon['striation_vecs']
                for x,y in pts_canvas.astype(np.int):
                    cv2.circle(viz, (x,y), 5, colors[label], -1)

                length = 50.
                ends = (pts_canvas + length * np.c_[vecs[:,1], -vecs[:,0]]).astype(np.int)
                for end, (x,y) in zip(ends, pts_canvas.astype(np.int)):
                    cv2.line(viz, (x,y), tuple(end), colors[label], thickness=2, lineType=8, shift=0)
                
    return viz

In [26]:
atlas_viz = visualize_atlas(landmark_polygons)
display(atlas_viz)

/oasis/projects/nsf/csd395/yuncong/Brain/notebooks/tmp.jpg

In [34]:
landmark_vertices = {}
for lm_ind in landmark_indices:
    print lm_ind
    
    with open('/home/yuncong/csd395/landmark_descriptors/RS141_0001_yuncong_08212015001655_landmark_descriptors_%d.pkl'%lm_ind, 'r') as f:
        landmark_polygons = pickle.load(f)[4]        
    landmark_vertices[lm_ind] = [landmark_polygons['boundary_vertices_global'], landmark_polygons['striation_points_global'], landmark_polygons['textured_area_vertices_global']]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [43]:
def visualize_atlas(landmark_vertices):
    '''
    Visualize the atlas with landmarks in original orientation
    '''
    
    canvas_size = np.load(os.environ['GORDON_RESULT_DIR'] + '/paddedSize.npy')
    origin_on_canvas = dm.load_pipeline_result('originPosOnCanvas', 'npy')
    
    viz = 255 * np.ones(np.r_[canvas_size, 3], np.uint8)
    viz[::20, :] = 0
    viz[:, ::20] = 0

    for label, (boundary_vertices, striation_vertices, textured_area_vertices) in landmark_vertices.iteritems():
        if len(boundary_vertices) > 0:
            pts_canvas = boundary_vertices + origin_on_canvas
            for x,y in pts_canvas.astype(np.int):
                cv2.circle(viz, (x,y), 5, colors[label], -1)
        
        if len(striation_vertices) > 0:
            pts_canvas = striation_vertices + origin_on_canvas
            for x,y in pts_canvas.astype(np.int):
                cv2.circle(viz, (x,y), 5, colors[label], -1)
        
        if len(textured_area_vertices) > 0:        
            pts_canvas = textured_area_vertices + origin_on_canvas
            for x,y in pts_canvas.astype(np.int):
                cv2.circle(viz, (x,y), 5, colors[label], -1)
                
    return viz

In [44]:
atlas_viz = visualize_atlas(landmark_vertices)
display(atlas_viz)

/oasis/projects/nsf/csd395/yuncong/Brain/notebooks/tmp.jpg

In [27]:
imsave(os.environ['GORDON_RESULT_DIR'] + '/atlas_viz.jpg', atlas_viz)

In [28]:
atlas_origin_on_canvas = DataManager(generate_hierarchy=False, stack='RS141', resol='x5', section=1).load_pipeline_result('originPosOnCanvas','npy')
np.save(os.environ['GORDON_RESULT_DIR'] + '/atlas_origin_on_canvas.npy', atlas_origin_on_canvas)

In [ ]:
# vis_overlay_alpha = np.ones(viz_overlay.shape[:2])
# vis_overlay_alpha[np.all(viz_overlay == 0, axis=-1)] = 0.

# viz = alpha_blending(dm.image_rgb, img_as_float(viz_overlay), .3, vis_overlay_alpha)
# # viz = alpha_blending(np.ones_like(dm.image_rgb), img_as_float(viz_overlay), .3, vis_overlay_alpha)
# display(viz)